# Instructions

- Selenium will need to login to facebook; enter your credentials in the "facebook_credentials.txt" file, in the "needed_files" folder

# Package import

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from collections import OrderedDict
from datetime import date
from pandas import DataFrame, ExcelWriter
%matplotlib inline

import time

# Login on facebook

In [2]:
# set options 
options = Options()
options.add_argument("--disable-infobars")
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_argument("--disable-notifications")

# obtaining email and password
with open('needed_files/facebook_credentials.txt', 'r') as file: # INSIRA AQUI O CAMINHO DO SEU ARQUIVO "facebook_credentials.txt"
    info = file.readline().split(';')
    EMAIL = info[0]
    PASSWORD = info[1]

# open facebook
s = Service('needed_files/chromedriver.exe') # INSIRA AQUI O CAMINHO DO SEU ARQUIVO "crhomedriver.exe"
driver = webdriver.Chrome(service=s, options=options)
driver.get('https://touch.facebook.com')
driver.maximize_window()
wait = WebDriverWait(driver, 30)

# insert email
email_field = wait.until(EC.visibility_of_element_located((By.NAME, 'email')))
email_field.send_keys(EMAIL)

# insert password
pass_field = wait.until(EC.visibility_of_element_located((By.NAME, 'pass')))
pass_field.send_keys(PASSWORD)

# login
pass_field.send_keys(Keys.RETURN)
time.sleep(5)

# click on "Not now" button
try:
    driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div[1]/div/div/div[3]/div[1]/div/div/a").click()
except:
    None


# Open selected post

In [3]:
# Open post
link = "https://touch.facebook.com/fordtrucks/photos/a.140763206189/10158648790831190/"
driver.get(link) 

# Extract posts comments and profiles links

In [5]:
extractedComments = []
try:
    Select(driver.find_element(By.TAG_NAME, "select")).select_by_index(index=1) # select "All comments" option
except:
    None
time.sleep(1)
try:
    for reply in driver.find_elements(By.CLASS_NAME, "_4ayk"): # open replies
        try:
            reply.click()
        except:
            None
except:
    None

time.sleep(1)

# Extract info
dataComments = driver.find_elements(By.CLASS_NAME, '_2b06')
for dc in dataComments:
    try:
        extractedComments.append('{}'.format(dc.text.split('\n')[1]))
    except:
        None

# Save obtained info

In [6]:
# remove duplicated
extractedComments = list(OrderedDict.fromkeys(extractedComments))

df = DataFrame(data=extractedComments, columns=['Comments'])

# defining file name
today = date.today()
fileName = 'facebook_extracted_info_{}.xlsx'.format(today.strftime("%b-%d-%Y"))

with ExcelWriter(fileName, engine='xlsxwriter') as writer:
    df.to_excel(writer,sheet_name='comments',index=False)